<a href="https://colab.research.google.com/github/jdmartinev/ArtificialIntelligenceIM/blob/main/Lecture06/notebooks/L06_VAE_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### [Autoencoders Variacionales (VAE)](https://arxiv.org/abs/1906.02691)

Los autoencoders tradicionales comprimen los datos de entrada en un espacio latente de menor dimensión, capturando efectivamente las características más importantes de las imágenes de entrada. Sin embargo, los vectores latentes producidos por los autoencoders tradicionales a menudo carecen de interpretabilidad. Por ejemplo, si consideramos el conjunto de datos MNIST, puede ser difícil entender qué dígitos corresponden a vectores latentes específicos porque los vectores latentes cercanos podrían no representar dígitos similares.

En contraste, cuando se entrenan modelos generativos, es ventajoso tener un espacio latente bien estructurado e interpretable. Aquí es donde entran en juego los **Autoencoders Variacionales (VAE)**.

Un VAE es un tipo de autoencoder que aprende a predecir una *distribución estadística* sobre los parámetros latentes, conocida como la **distribución latente**. Por ejemplo, en un VAE, podríamos suponer que los vectores latentes siguen una distribución Gaussiana

$$
N(\mu_{\mathbf{z}}, e^{\log \sigma_{\mathbf{z}}})
$$

donde $\mu_{\mathbf{z}}$ y $\log \sigma_{\mathbf{z}} \in \mathbb{R}^d$. El codificador en un VAE aprende a predecir estos parámetros de la distribución, y el decodificador reconstruye la entrada muestreando un vector de esta distribución.

Para resumir el proceso de un VAE:

1. A partir de los datos de entrada, el codificador predice $\mu_{\mathbf{z}}$ y $\log \sigma_{\mathbf{z}}$ (predecimos el logaritmo de la desviación estándar en lugar de la desviación estándar en sí por estabilidad numérica).
2. Luego, muestreamos un vector latente $\mathbf{z}_{\text{sample}}$ de la distribución

$$
N(\mu_{\mathbf{z}}, e^{\log \sigma_{\mathbf{z}}})
$$

3. El decodificador intenta reconstruir la entrada original utilizando $\mathbf{z}_\text{sample}}$ como entrada para la red del decodificador.

Este proceso permite que el VAE aprenda un espacio latente suave y continuo donde la interpolación entre puntos en el espacio latente resulta en variaciones significativas en la salida. Esta propiedad hace que los VAE sean particularmente poderosos para generar nuevos datos que son similares a los datos de entrenamiento.

Aquí tienes una representación visual de la arquitectura del VAE:

![Imagen de la arquitectura del VAE](https://drive.google.com/uc?id=1x30qjCzLYIuJJR_dBFDHGolpltYLCCP2)

> Imagen tomada de [este blog](https://ijdykeman.github.io/ml/2016/12/21/cvae.html) de Isaak Dykeman


In [ ]:
import torch
import torchvision
import matplotlib.pyplot as plt
from torchvision import transforms
from torch import nn
from torch import optim
from tqdm import tqdm
import numpy as np
import torch.nn.functional as F
torch.manual_seed(42)
np.random.seed(42)

In [ ]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
train_size = 0.9
lr = 1e-3
eps = 1e-8
batch_size = 256
epochs = 30

In [ ]:
def mnist(train_part, transform=None):
    dataset = torchvision.datasets.MNIST('.', download=True, transform=transform)
    train_part = int(train_part * len(dataset))
    train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_part, len(dataset) - train_part])
    return train_dataset, test_dataset

In [ ]:
transform = transforms.Compose([transforms.ToTensor()])

train_dataset, test_dataset = mnist(train_size, transform)

train_dataloader = torch.utils.data.DataLoader(train_dataset, drop_last=True, batch_size=batch_size, shuffle=True)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=1, shuffle=False)
dataloaders = (train_dataloader, test_dataloader)

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9912422/9912422 [00:00<00:00, 25451390.20it/s]


Extracting ./MNIST/raw/train-images-idx3-ubyte.gz to ./MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28881/28881 [00:00<00:00, 1714588.73it/s]

Extracting ./MNIST/raw/train-labels-idx1-ubyte.gz to ./MNIST/raw



Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1648877/1648877 [00:00<00:00, 14663962.68it/s]


Extracting ./MNIST/raw/t10k-images-idx3-ubyte.gz to ./MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4542/4542 [00:00<00:00, 4063679.34it/s]


Extracting ./MNIST/raw/t10k-labels-idx1-ubyte.gz to ./MNIST/raw



In [ ]:
def plotn(n, data, noisy=False, super_res=None):
    fig, ax = plt.subplots(1, n)
    for i, z in enumerate(data):
        if i == n:
            break
        preprocess = z[0].reshape(1, 28, 28) if z[0].shape[1] == 28 else z[0].reshape(1, 14, 14) if z[0].shape[1] == 14 else z[0]
        if super_res is not None:
            _transform = transforms.Resize((int(preprocess.shape[1] / super_res), int(preprocess.shape[2] / super_res)))
            preprocess = _transform(preprocess)

        if noisy:
            shapes = list(preprocess.shape)
            preprocess += noisify(shapes)

        ax[i].imshow(preprocess[0])
    plt.show()

In [ ]:
def noisify(shapes):
    return np.random.normal(loc=0.5, scale=0.3, size=shapes)

In [ ]:
class VAEEncoder(nn.Module):
    def __init__(self, device):
        super().__init__()
        self.intermediate_dim = 512
        self.latent_dim = 2
        self.linear = nn.Linear(784, self.intermediate_dim)
        self.z_mean = nn.Linear(self.intermediate_dim, self.latent_dim)
        self.z_log = nn.Linear(self.intermediate_dim, self.latent_dim)
        self.relu = nn.ReLU()
        self.device = device

    def forward(self, input):
        bs = input.shape[0]

        hidden = self.relu(self.linear(input))
        z_mean = self.z_mean(hidden)
        z_log = self.z_log(hidden)

        eps = torch.FloatTensor(np.random.normal(size=(bs, self.latent_dim))).to(device)
        z_val = z_mean + torch.exp(z_log) * eps
        return z_mean, z_log, z_val

In [ ]:
class VAEDecoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.intermediate_dim = 512
        self.latent_dim = 2
        self.linear = nn.Linear(self.latent_dim, self.intermediate_dim)
        self.output = nn.Linear(self.intermediate_dim, 784)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, input):
        hidden = self.relu(self.linear(input))
        decoded = self.sigmoid(self.output(hidden))
        return decoded

In [ ]:
class VAEAutoEncoder(nn.Module):
    def __init__(self, device):
        super().__init__()
        self.encoder = VAEEncoder(device)
        self.decoder = VAEDecoder()
        self.z_vals = None

    def forward(self, input):
        bs, c, h, w = input.shape[0], input.shape[1], input.shape[2], input.shape[3]
        input = input.view(bs, -1)
        encoded = self.encoder(input)
        self.z_vals = encoded
        decoded = self.decoder(encoded[2])
        return decoded

    def get_zvals(self):
        return self.z_vals

Los autoencoders variacionales utilizan una función de pérdida compleja que consta de dos partes:

* **Pérdida de reconstrucción**: es la función de pérdida que muestra qué tan cercana es la imagen reconstruida a la original (puede ser MSE). Es la misma función de pérdida que se utiliza en los autoencoders normales.
* **Pérdida KL**, que asegura que las distribuciones de las variables latentes se mantengan cercanas a una distribución normal. Se basa en la noción de la [divergencia de Kullback-Leibler](https://www.countbayesie.com/blog/2017/5/9/kullback-leibler-divergence-explained), una métrica para estimar cuán similares son dos distribuciones estadísticas.


In [ ]:
def vae_loss(preds, targets, z_vals):
    mse = nn.MSELoss()
    reconstruction_loss = mse(preds, targets.view(targets.shape[0], -1)) * 784.0
    temp = 1.0 + z_vals[1] - torch.square(z_vals[0]) - torch.exp(z_vals[1])
    kl_loss = -0.5 * torch.sum(temp, axis=-1)
    return torch.mean(reconstruction_loss + kl_loss)

In [ ]:
model = VAEAutoEncoder(device).to(device)
optimizer = optim.RMSprop(model.parameters(), lr=lr, eps=eps)

In [ ]:
def train_vae(dataloaders, model, optimizer, epochs, device):
    tqdm_iter = tqdm(range(epochs))
    train_dataloader, test_dataloader = dataloaders[0], dataloaders[1]

    for epoch in tqdm_iter:
        model.train()
        train_loss = 0.0
        test_loss = 0.0

        for batch in train_dataloader:
            imgs, labels = batch
            imgs = imgs.to(device)
            labels = labels.to(device)

            preds = model(imgs)
            z_vals = model.get_zvals()
            loss = vae_loss(preds, imgs, z_vals)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            train_loss += loss.item()

        model.eval()
        with torch.no_grad():
            for batch in test_dataloader:
                imgs, labels = batch
                imgs = imgs.to(device)
                labels = labels.to(device)

                preds = model(imgs)
                z_vals = model.get_zvals()
                loss = vae_loss(preds, imgs, z_vals)

                test_loss += loss.item()

        train_loss /= len(train_dataloader)
        test_loss /= len(test_dataloader)

        tqdm_dct = {'train loss:': train_loss, 'test loss:': test_loss}
        tqdm_iter.set_postfix(tqdm_dct, refresh=True)
        tqdm_iter.refresh()

In [ ]:
train_vae(dataloaders, model, optimizer, epochs, device)

100%|██████████| 30/30 [06:18<00:00, 12.62s/it, train loss:=35.2, test loss:=35.8]


In [ ]:
model.eval()
predictions = []
plots = 5
for i, data in enumerate(test_dataset):
    if i == plots:
        break
    predictions.append(model(data[0].to(device).unsqueeze(0)).view(1, 28, 28).detach().cpu())
plotn(plots, test_dataset)
plotn(plots, predictions)

> **Tarea**: En nuestro ejemplo, hemos entrenado un VAE completamente conectado (fully-connected). Ahora toma la CNN del autoencoder tradicional mencionado anteriormente y crea un VAE basado en CNN.


# Materiales Adicionales

* [Explicación VAE](https://mbernste.github.io/posts/vae/)
